# Load packages

In [25]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras import backend
from tensorflow.keras.layers import Dense, Dropout, Flatten, MaxPool2D, Conv2D
from tensorflow.keras.models import Sequential
import time

# Load and prepare data

In [2]:
# Load saved numpy arrays
X = np.load('/Users/Donovan/Documents/Masters/masters-ed02/coding/model_base_cnn/train/X.npy')
Y = np.load('/Users/Donovan/Documents/Masters/masters-ed02/coding/model_base_cnn/train/Y.npy')

In [9]:
# Subsample
sample_size = 1000

g_sample = int(np.floor(sample_size/2))
n_sample = sample_size - g_sample

g_i = np.random.randint(0, int(X.shape[0]/2) - 1, g_sample)
n_i = np.random.randint(int(X.shape[0]/2), X.shape[0] - 1, n_sample)

In [21]:
X = np.concatenate([X[g_i], X[n_i]])
Y = np.concatenate([Y[g_i], Y[n_i]])

In [22]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.20, random_state=42, shuffle = True)

In [23]:
print ('X_train:',X_train.shape)
print ('Y_train:',Y_train.shape)
print ()
print ('X_val:',X_val.shape)
print ('Y_val:',Y_val.shape)

X_train: (800, 128, 188, 1)
Y_train: (800, 2)

X_val: (200, 128, 188, 1)
Y_val: (200, 2)


# Basic 2D CNN model

In [24]:
def network_2D(conv_layers, fc_layers, conv_filters, dropout_rate, 
               conv_kernel,max_pooling_size,fc_units,epochs,batch_size):
    
    model = Sequential()
    model.add(Conv2D(filters = conv_filters, kernel_size = conv_kernel, input_shape = (128,188,1)
                     ,activation = 'relu'))
    model.add(Dropout(rate = dropout_rate))
    model.add(MaxPool2D(pool_size = max_pooling_size))
    
    for i in range(conv_layers):
        model.add(Conv2D(filters = conv_filters, kernel_size = conv_kernel, activation = 'relu'))
        model.add(Dropout(rate=dropout_rate))
        model.add(MaxPool2D(pool_size=max_pooling_size))
        
    model.add(Flatten())
    for i in range(fc_layers):
        model.add(Dense(units = fc_units, activation='relu'))
        model.add(Dropout(rate=dropout_rate))
        
    model.add(Dense(2, activation = 'softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
    
    return model

def network():

    conv_layers = 1
    fc_layers = 1
    max_pooling_size = 4
    dropout_rate = 0.4
    conv_filters = 8
    conv_kernel = 16
    fc_units = 32
    epochs = 40
    batch_size = 8
    model = network_2D(conv_layers, fc_layers, conv_filters, dropout_rate, 
               conv_kernel,max_pooling_size,fc_units,epochs,batch_size)
    
    return model

# Train model

In [26]:
model = network()
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

model.summary()

start = time.time()

history = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), 
            batch_size=8,
            epochs=50,
            verbose=2, 
            class_weight={0:1.,1:1.})
end = time.time()

2022-10-12 16:57:42.219800: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-10-12 16:57:42.220170: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 113, 173, 8)       2056      
                                                                 
 dropout (Dropout)           (None, 113, 173, 8)       0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 28, 43, 8)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 28, 8)         16392     
                                                                 
 dropout_1 (Dropout)         (None, 13, 28, 8)         0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 3, 7

2022-10-12 16:57:43.057820: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-10-12 16:57:43.386257: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


100/100 - 5s - loss: 0.6438 - accuracy: 0.6287 - val_loss: 0.6388 - val_accuracy: 0.7850 - 5s/epoch - 51ms/step
Epoch 2/50


2022-10-12 16:57:47.953975: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


100/100 - 2s - loss: 0.5219 - accuracy: 0.7825 - val_loss: 0.5488 - val_accuracy: 0.8700 - 2s/epoch - 17ms/step
Epoch 3/50
100/100 - 2s - loss: 0.4318 - accuracy: 0.8212 - val_loss: 0.4679 - val_accuracy: 0.8550 - 2s/epoch - 17ms/step
Epoch 4/50
100/100 - 2s - loss: 0.4254 - accuracy: 0.8312 - val_loss: 0.4685 - val_accuracy: 0.8850 - 2s/epoch - 17ms/step
Epoch 5/50
100/100 - 2s - loss: 0.3673 - accuracy: 0.8550 - val_loss: 0.4198 - val_accuracy: 0.8950 - 2s/epoch - 17ms/step
Epoch 6/50
100/100 - 2s - loss: 0.3145 - accuracy: 0.8700 - val_loss: 0.4054 - val_accuracy: 0.9200 - 2s/epoch - 17ms/step
Epoch 7/50
100/100 - 2s - loss: 0.2895 - accuracy: 0.8912 - val_loss: 0.4047 - val_accuracy: 0.8600 - 2s/epoch - 16ms/step
Epoch 8/50
100/100 - 2s - loss: 0.2562 - accuracy: 0.8850 - val_loss: 0.3298 - val_accuracy: 0.9050 - 2s/epoch - 17ms/step
Epoch 9/50
100/100 - 2s - loss: 0.2314 - accuracy: 0.9087 - val_loss: 0.3081 - val_accuracy: 0.9000 - 2s/epoch - 17ms/step
Epoch 10/50
100/100 - 2s - 